In [9]:
import numpy as np
from math import log
import itertools
from skimage.feature import peak_local_max 
from scipy import ndimage
import skimage.io as io
from skimage import filters
import itertools as itt
import preprocessor
from math import log
from os import listdir
import subprocess
from os.path import isfile, join
import math

In [10]:
"""
Detect somata in janelia images

"""

'\nDetect somata in janelia images\n\n'

In [11]:
mypath = '/home/yingtao/Desktop/annote3D/'
desktop = '/home/yingtao/Desktop/'

In [12]:
##old fast function to prune skimage detected result
def _prune_fast(blobs, overlap, ratio):
    newblobs = [p for i, p in enumerate(blobs) if all(_distance(k, p, ratio)>overlap for k in blobs[i+1:])]
    return newblobs

In [13]:
##old functions to prune skimage detected result
#Prune close points
def _distance(blob1, blob2, ratio):
    dif = np.array([(blob1[0]-blob2[0])*ratio, blob1[1]-blob2[1],  blob1[2]-blob2[2]])
    dis = np.linalg.norm(dif)
    return dis
def _prune_blobs(blobs, overlap, ratio):
    for blob1, blob2 in itt.combinations(blobs, 2):
        if _distance(blob1, blob2, ratio) < overlap:
            if blob1[3] > blob2[3]:
                blob2[3] = 0
            else:
                blob1[3] = 0
    return blobs

In [14]:
def getboxslice(image,center,xd,yd):
    center = center.astype('int')
    try:
        temp = image[center[0] - 1,
                     center[2]-xd//2:center[2]+xd//2+1, 
                     center[1]-yd//2:center[1]+yd//2+1,
                     ]
        if temp.size == xd*yd:
            return temp
        else:
            return 0 
    except IndexError:
        return 0
def _discenter(x,y,xd,yd):
    dis = (x - xd//2) ** 2 / (xd//2) ** 2  + (y - yd//2) ** 2 / (yd//2) ** 2
    if  0.2 < dis < 0.8:
        return 1
    else:
        return 0
#return a ellipse sphere kernal
def addvalue(dog_list, image):
    xd, yd = 21, 21
    temp = [_discenter(x, y, xd, yd) for x in range(xd) for y in range(yd)]
    coin_kernel = np.array(temp).reshape(xd,yd)
    pixelvalue = np.array([np.sum(coin_kernel * getboxslice(image, i, xd, yd)) for i in dog_list])
    return np.concatenate([dog_list,pixelvalue.reshape(-1,1)],axis=1)
def _distance(blob1, blob2):
    dif = np.array([blob1[1]-blob2[1],  blob1[2]-blob2[2]])
    dis = np.linalg.norm(dif)
    return dis
def _prune_blobs(blobs, overlap):
    for i, blob1 in enumerate(blobs):
        k = blob1[0]
        for blob2 in blobs[i:,]:
            if blob2[0] - k > 2.5:
                break
            if _distance(blob1, blob2) < overlap and 0 < abs(blob1[0] - blob2[0]) < 3:
                if blob1[3] > blob2[3]:
                    blob2[3] = 0
                else:
                    blob1[3] = 0
    blobs = np.array([blob for blob in blobs if blob[3] != 0])
    return blobs

In [15]:
#calculate dog, detect peaks
def dog_maxima(terminal, sigma, zx_ratio=4, blur_sigma=0.3, size = 8, prune_dis = 2):
    s = sigma
    gaussian_images0 = ndimage.gaussian_filter(terminal, (s/4, s,s))
    gaussian_images1 = ndimage.gaussian_filter(terminal, ((s*1.3)/4, s*1.3,s*1.3))
    dog_image = np.abs((gaussian_images1* 1.3 ** 3).astype('int16') - gaussian_images0.astype("int16"))
    dog_list = getmaxima(dog_image)
    
    #blurred = ndimage.filters.gaussian_filter(dog_image.astype("int16"), sigma=(blur_sigma/zx_ratio, blur_sigma, blur_sigma))
    #local_maxima = peak_local_max(dog_image, min_distance=5, threshold_rel=0.05, exclude_border=True, num_peaks_per_label==1000)#, threshold_abs=s, footprint=np.ones([size//zx_ratio + 1, size, size]))
    #print('a')
    #dog_list = [1,1]
    #dog_list = _prune_fast(local_maxima, prune_dis, zx_ratio)
    return dog_list, np.swapaxes(dog_image,1,2) #dog_list_pruned, dog_images, blurred

In [16]:
#detect peaks by imagej function

def getfiles(filepath):
    files = [join(filepath, f) for f in listdir(filepath) if isfile(join(filepath, f))]
    files.sort()
    return files
def getlsmfiles(filepath):
    files = [join(filepath, f) for f in listdir(filepath) if f[-3:] == 'lsm']
    files.sort()
    return files
def getmaxima(dog_image):
    io.imsave(mypath +'original_images/1106.tif', dog_image.astype('int16'))
    p = subprocess.Popen(['xvfb-run','-a', desktop + "Fiji.app/./ImageJ-linux64", "-macro",
                          mypath + "macros/findmaxima.ijm"])
    p.wait()
    onlyfiles = getfiles(mypath + "maximafile")
    maxima_data = np.empty([0,3])
    for i, files in enumerate(onlyfiles):
        temp = np.genfromtxt(files, delimiter=',')
        try:
            temp[:, 0] = i + 1
        except IndexError:
            continue           
        maxima_data = np.append(maxima_data, temp, axis = 0)
    dog_list = np.array([i for i in maxima_data if not math.isnan(i[1])])
    p = subprocess.Popen("rm "+ mypath + "maximafile/*", shell = True)
    #order of coordinate z,x,y
    return dog_list

In [17]:
#prepare sample
def samplefromindex(dog_list, positive_labels, image, xysize, sample_size, xz_ratio):
    samples = np.empty([0, sample_size])
    labels = []
    for i, coord in enumerate(dog_list):
        coord = coord.astype(int)
        if coord[0] < xysize//xz_ratio:
            coord[0] = xysize//xz_ratio
        elif coord[0] > image.shape[0] - xysize//xz_ratio - 0.5:
            coord[0] = image.shape[0] - xysize//xz_ratio - 1
        if coord[1] < xysize + 0.5:
            coord[1] = xysize
        elif coord[1] > image.shape[2] - xysize - 0.5:
            coord[1] = image.shape[2] - xysize - 1
        if coord[2]< xysize + 0.5:
            coord[2] = xysize
        elif coord[2] > image.shape[1] - xysize - 0.5:
            coord[2] = image.shape[1] - xysize - 1
        coord_slice = np.index_exp[coord[0]-xysize//xz_ratio:coord[0]+xysize//xz_ratio+1, 
                                      coord[2]-xysize:coord[2]+xysize+1,  coord[1]-xysize:coord[1]+xysize+1]
        temp = image[coord_slice].flatten()
        if temp.shape[0] == samples.shape[1]:
            temp = temp.reshape(1,-1)
            samples = np.append(samples, temp, axis=0)
        else:
            print(i)
            continue
        if i in positive_labels:
            labels.append(1)
        else:
            labels.append(0)
    return samples, labels

In [29]:
#batch process
def batch_label(path = 'default path'):
    if path == 'default path':
        path = '/home/yingtao/Desktop/annote3D/' + "janelia_gal4"
    labels = []
    plabels_indiv = []
    samples_indiv = []
    sample_xysize = 15
    xz_ratio = 4
    sample_size = (sample_xysize//xz_ratio * 2 + 1)*(sample_xysize*2 + 1)**2
    samples = np.empty([0, sample_size])
    janl_lsms = getlsmfiles(mypath + "janelia_gal4")
    stopflag = 0
    for i, janl_lsm in enumerate(janl_lsms):
        if stopflag == 1:
            break
        image, origin_image = preprocessor.prepro(janl_lsm)
        dog_list, dog_image = dog_maxima(image, 2, size =7)
        dog_list = addvalue(dog_list, origin_image)
        dog_list = _prune_blobs(dog_list, 10)
        np.savetxt(mypath + 'dog_maxima', dog_list)
        io.imsave(mypath + 'origin.tif', origin_image.astype("int16"))
        p = subprocess.Popen([desktop + "Fiji.app/./ImageJ-linux64", "-macro",
                          mypath + "macros/showcirc.ijm"])
        for temp in dog_list:
            s = input("What's the index of circled neuron? ")
            if s == 'e':
                print('next image')
                samples_indiv, labels_indiv = samplefromindex(dog_list, plabels_indiv, 
                                                                          origin_image, sample_xysize, sample_size, xz_ratio)
                samples = np.append(samples, samples_indiv, axis = 0)
                labels.extend(labels_indiv)
                plabels_indiv =[]
                break
            if s == 'd':
                print('deleteone')
                plabels_indiv.pop()
            if s == 'n':
                print('neglect it')
                break
            if s == 'stop':
                print('next image')
                samples_indiv, labels_indiv = samplefromindex(dog_list, plabels_indiv, 
                                                                          origin_image, sample_xysize, sample_size, xz_ratio)
                samples = np.append(samples, samples_indiv, axis = 0)
                labels.extend(labels_indiv)
                plabels_indiv =[]
                stopflag = 1
                break
            try:
                plabels_indiv.append(int(s))
            except ValueError:
                print('do it again!')
            print("neuron + " +s+"")
        np.savetxt(mypath + 'labeled_samples_file/' + str(i) + 'samples.txt', samples_indiv)
        np.savetxt(mypath + 'labeled_samples_file/' + str(i) + 'labels.txt', labels_indiv)
    return samples, labels, i
#batch process
def batch_test(path = 'default path'):
    if path == 'default path':
        path = '/home/yingtao/Desktop/annote3D/' + "janelia_gal4"
    labels = []
    plabels_indiv = []
    samples_indiv = []
    sample_xysize = 15
    xz_ratio = 4
    sample_size = (sample_xysize//xz_ratio * 2 + 1)*(sample_xysize*2 + 1)**2
    samples = []
    janl_lsms = getlsmfiles(mypath + "janelia_gal4")
    stopflag = 0
    for i, janl_lsm in enumerate(janl_lsms):
        if stopflag == 1:
            break
        if i == 0:
            print('next')
            continue
        image, origin_image = preprocessor.prepro(janl_lsm)
        dog_list, dog_image = dog_maxima(image, 2, size =7)
        dog_list = addvalue(dog_list, origin_image)
        dog_list = _prune_blobs(dog_list, 10)
        plabels_indiv = np.ones(dog_list.shape[0])
        samples_indiv, labels_indiv = samplefromindex(dog_list, plabels_indiv, 
                                                                          origin_image, sample_xysize, sample_size, xz_ratio)
        samples.append(samples_indiv)
        np.savetxt(mypath + 'test_files/' + str(i) + 'samples.txt', samples_indiv)
        s = input("Do you want to stop?(y/n)")
        if s == 'y' or 'Y':
            break
    return samples

0

In [25]:
temp1, temp2 = batch_test()

next
Init 1/3 with method: k-means++
Inertia for init 1/3: 21105765.277652
Init 2/3 with method: k-means++
Inertia for init 2/3: 19139881.515201
Init 3/3 with method: k-means++
Inertia for init 3/3: 21671286.922006
Minibatch iteration 1/3800: mean batch inertia: 6.243028, ewa inertia: 6.243028 
Minibatch iteration 2/3800: mean batch inertia: 6.452349, ewa inertia: 6.254322 
Minibatch iteration 3/3800: mean batch inertia: 6.343007, ewa inertia: 6.259107 
Minibatch iteration 4/3800: mean batch inertia: 6.376764, ewa inertia: 6.265456 
Minibatch iteration 5/3800: mean batch inertia: 6.375967, ewa inertia: 6.271419 
Minibatch iteration 6/3800: mean batch inertia: 6.298362, ewa inertia: 6.272873 
Minibatch iteration 7/3800: mean batch inertia: 6.438154, ewa inertia: 6.281791 
Minibatch iteration 8/3800: mean batch inertia: 6.383038, ewa inertia: 6.287254 
Minibatch iteration 9/3800: mean batch inertia: 6.371811, ewa inertia: 6.291817 
Minibatch iteration 10/3800: mean batch inertia: 6.26854

/usr/local/lib/python3.5/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: /home/yingtao/Desktop/annote3D/image.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: /home/yingtao/Desktop/annote3D/original_images/1106.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: UserWarning: genfromtxt: Empty input file: "/home/yingtao/Desktop/annote3D/maximafile/Results68.csv"
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: UserWarning: genfromtxt: Empty input file: "/home/yingtao/Desktop/annote3D/maximafile/Results69.csv"
/usr/local/lib/python3.5/dist-packages/ipy

Do you want to stop?(y/n)y


In [30]:
#create labelled images. Image and labels is shown in imagej
labels = []
plabels_indiv = []
samples_indiv = []
sample_xysize = 15
xz_ratio = 4
sample_size = (sample_xysize//xz_ratio * 2 + 1)*(sample_xysize*2 + 1)**2
samples = np.empty([0, sample_size])
janl_lsms = getlsmfiles(mypath + "janelia_gal4")

for i, janl_lsm in enumerate(janl_lsms):
    if i == 0:
        print('next')
        continue
    image, origin_image = preprocessor.prepro(janl_lsm)
    dog_list, dog_image = dog_maxima(image, 2, size =7)
    dog_list = addvalue(dog_list, origin_image)
    dog_list = _prune_blobs(dog_list, 10)
    np.savetxt(mypath + 'dog_maxima', dog_list)
    io.imsave(mypath + 'origin.tif', origin_image.astype("int16"))
    p = subprocess.Popen([desktop + "Fiji.app/./ImageJ-linux64", "-macro",
                      mypath + "macros/showcirc.ijm"])
    for temp in dog_list:
        s = input("What's the index of circled neuron? ")
        if s == 'e':
            print('next image')
            samples_indiv, labels_indiv = samplefromindex(dog_list, plabels_indiv, 
                                                                      origin_image, sample_xysize)
            samples = np.append(samples, samples_indiv, axis = 0)
            labels.extend(labels_indiv)
            plabels_indiv =[]
            break
        if s == 'd':
            print('deleteone')
            plabels_indiv.pop()
        if s == 'n':
            print('neglect it')
            break
        try:
            plabels_indiv.append(int(s))
        except ValueError:
            print('do it again!')
        print("neuron + " +s+"")
    np.savetxt(mypath + 'labeled_samples_file/' + str(i) + 'samples.txt', samples_indiv)
    np.savetxt(mypath + 'labeled_samples_file/' + str(i) + 'labels.txt', labels_indiv)
    

next
Init 1/3 with method: k-means++
Inertia for init 1/3: 20716132.223799
Init 2/3 with method: k-means++
Inertia for init 2/3: 19501366.023899
Init 3/3 with method: k-means++
Inertia for init 3/3: 19776458.786843
Minibatch iteration 1/3800: mean batch inertia: 6.467167, ewa inertia: 6.467167 
Minibatch iteration 2/3800: mean batch inertia: 6.481267, ewa inertia: 6.467928 
Minibatch iteration 3/3800: mean batch inertia: 6.398649, ewa inertia: 6.464190 
Minibatch iteration 4/3800: mean batch inertia: 6.485213, ewa inertia: 6.465324 
Minibatch iteration 5/3800: mean batch inertia: 6.430606, ewa inertia: 6.463451 
Minibatch iteration 6/3800: mean batch inertia: 6.431915, ewa inertia: 6.461749 
Minibatch iteration 7/3800: mean batch inertia: 6.409712, ewa inertia: 6.458941 
Minibatch iteration 8/3800: mean batch inertia: 6.558686, ewa inertia: 6.464323 
Minibatch iteration 9/3800: mean batch inertia: 6.451624, ewa inertia: 6.463638 
Minibatch iteration 10/3800: mean batch inertia: 6.31065

/usr/local/lib/python3.5/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: /home/yingtao/Desktop/annote3D/image.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: /home/yingtao/Desktop/annote3D/original_images/1106.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: UserWarning: genfromtxt: Empty input file: "/home/yingtao/Desktop/annote3D/maximafile/Results68.csv"
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: UserWarning: genfromtxt: Empty input file: "/home/yingtao/Desktop/annote3D/maximafile/Results69.csv"
/usr/local/lib/python3.5/dist-packages/ipy

KeyboardInterrupt: 

In [3]:
#divide learning and test samples
from sklearn.model_selection import StratifiedKFold
skf  = StratifiedKFold(n_splits=3)
for a,b in skf.split(samples, labels):
    trainindex = a
    testindex = b
    break

NameError: name 'samples' is not defined

In [ ]:
skf  = StratifiedKFold

In [251]:
#prepare randomforest classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5, random_state=0, class_weight = 'balanced')

In [260]:
#learning
clf.fit(samples[trainindex], np.array(labels)[trainindex])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [262]:
#score
clf.score(samples[trainindex], np.array(labels)[trainindex])

0.9853479853479854

In [263]:
#score
scores = cross_val_score(clf, samples, np.array(labels), cv = 5)

next
Init 1/3 with method: k-means++
Inertia for init 1/3: 21067357.713138
Init 2/3 with method: k-means++
Inertia for init 2/3: 20376546.486481
Init 3/3 with method: k-means++
Inertia for init 3/3: 19010382.633923
Minibatch iteration 1/3800: mean batch inertia: 6.343149, ewa inertia: 6.343149 
Minibatch iteration 2/3800: mean batch inertia: 6.384354, ewa inertia: 6.345372 
Minibatch iteration 3/3800: mean batch inertia: 6.328353, ewa inertia: 6.344454 
Minibatch iteration 4/3800: mean batch inertia: 6.350746, ewa inertia: 6.344793 
Minibatch iteration 5/3800: mean batch inertia: 6.220066, ewa inertia: 6.338063 
Minibatch iteration 6/3800: mean batch inertia: 6.194481, ewa inertia: 6.330316 
Minibatch iteration 7/3800: mean batch inertia: 6.284986, ewa inertia: 6.327870 
Minibatch iteration 8/3800: mean batch inertia: 6.249867, ewa inertia: 6.323661 
Minibatch iteration 9/3800: mean batch inertia: 6.260149, ewa inertia: 6.320234 
Minibatch iteration 10/3800: mean batch inertia: 6.26004

/usr/local/lib/python3.5/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: /home/yingtao/Desktop/annote3D/image.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: /home/yingtao/Desktop/annote3D/original_images/1106.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: UserWarning: genfromtxt: Empty input file: "/home/yingtao/Desktop/annote3D/maximafile/Results68.csv"
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: UserWarning: genfromtxt: Empty input file: "/home/yingtao/Desktop/annote3D/maximafile/Results69.csv"
/usr/local/lib/python3.5/dist-packages/ipy

What's the index of circled neuron? 1
neuron + 1
What's the index of circled neuron? 2
neuron + 2
What's the index of circled neuron? 3
neuron + 3
What's the index of circled neuron? d
deleteone
do it again!
neuron + d
What's the index of circled neuron? e
next image
Init 1/3 with method: k-means++
Inertia for init 1/3: 37876371.389689
Init 2/3 with method: k-means++
Inertia for init 2/3: 43345467.125200
Init 3/3 with method: k-means++
Inertia for init 3/3: 36463925.950470
Minibatch iteration 1/4000: mean batch inertia: 12.327134, ewa inertia: 12.327134 
Minibatch iteration 2/4000: mean batch inertia: 12.250051, ewa inertia: 12.323226 
Minibatch iteration 3/4000: mean batch inertia: 12.130358, ewa inertia: 12.313450 
Minibatch iteration 4/4000: mean batch inertia: 12.099442, ewa inertia: 12.302601 
Minibatch iteration 5/4000: mean batch inertia: 12.073313, ewa inertia: 12.290978 
Minibatch iteration 6/4000: mean batch inertia: 12.190035, ewa inertia: 12.285861 
Minibatch iteration 7/40

/usr/local/lib/python3.5/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: /home/yingtao/Desktop/annote3D/original_images/1106.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: /home/yingtao/Desktop/annote3D/origin.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


What's the index of circled neuron? 2
neuron + 2
What's the index of circled neuron? 3
neuron + 3
What's the index of circled neuron? 4
neuron + 4
What's the index of circled neuron? stop


In [34]:
temp1

array([[ 6.,  3.,  3., ...,  2.,  2.,  2.],
       [ 2.,  2.,  3., ...,  3.,  3.,  5.],
       [ 2.,  3.,  2., ...,  2.,  3.,  4.],
       ..., 
       [ 2.,  2.,  5., ...,  2.,  2.,  1.],
       [ 2.,  2.,  2., ...,  2.,  2.,  2.],
       [ 2.,  2.,  2., ...,  2.,  2.,  2.]])

In [35]:
temp2

[0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
